In [1]:
import requests
import psycopg2
import json
import arcpy


# Make a GET request to the URL and parse the response as JSON
url = 'https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Planning_Primary_Zoning/FeatureServer/0/query?outFields=ZONE_CODE&where=1%3D1&f=geojson'
response = requests.get(url)
data = json.loads(response.content)

# Find overlapping polygons

In [7]:
# set the input URL. Different then one from above
input_url = 'https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Planning_Primary_Zoning/FeatureServer/0'

# create a feature layer from the input URL
arcpy.MakeFeatureLayer_management(input_url, "input_layer8")

# find overlapping polygons with the intersect tool
output_layer8 = arcpy.Intersect_analysis(["input_layer8", "input_layer8"], "memory\\output_layer8")

# print the number of features in the output layer
print(arcpy.GetCount_management(output_layer8))

# create a dictionary to store the overlapping features
overlapping_features = {}

# loop through the output layer and populate the dictionary
with arcpy.da.SearchCursor(output_layer8, ['OID@', 'SHAPE@']) as cursor:
    for row in cursor:
        oid, shape = row
        if oid not in overlapping_features:
            overlapping_features[oid] = [shape]
        else:
            overlapping_features[oid].append(shape)

# loop through the overlapping features and print their object IDs
for oid, shapes in overlapping_features.items():
    if len(shapes) > 1:
        print(oid)

# delete the output layer from memory
arcpy.Delete_management(output_layer8)

2215


<Result 'true'>

# Find null values in data and print errors to an error code column

In [5]:
features = data['features']
    
conn = psycopg2.connect(database="lab0", user="postgres", password="", host="34.27.240.213")
cur = conn.cursor()

for feature in features:
    zone_code = feature['properties']['ZONE_CODE']
    geometry = json.dumps(feature['geometry'])
    cur.execute("SELECT ST_AsText(ST_GeomFromGeoJSON(%s))", (geometry,))
    wkt_geometry = cur.fetchone()[0]

    # Check for null values in zone_code and wkt_geometry
    error_code = None
    if zone_code is None:
        error_code = 'ZoneError'
    elif wkt_geometry is None:
        error_code = 'GeomError'
    
    cur.execute("INSERT INTO MPLSZoning (zone_code, geometry, error_code) VALUES (%s, %s, %s)", (zone_code, wkt_geometry, error_code))
    conn.commit()

cur.close()
conn.close()


KeyboardInterrupt: 